Trying to create "variable" model that would take variable lenght of data- impossible

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from itertools import chain
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
#import keras_tuner as kt
df = pd.read_csv("drive/MyDrive/Engineer's Project/test.csv")

In [ ]:
df.pop('usa_cpi')
df.pop('pol_cpi')
df.pop('usa_inter')
df.pop('pol_inter')
df.pop('Date')

0       2010.11.15
1       2010.11.16
2       2010.11.17
3       2010.11.18
4       2010.11.19
           ...    
3537    2022.03.27
3538    2022.03.28
3539    2022.03.29
3540    2022.03.30
3541    2022.03.31
Name: Date, Length: 3542, dtype: object

In [ ]:
scaler = MinMaxScaler()
df_numpy = df.to_numpy() 
scaler = scaler.fit(df_numpy[2000:3101])
df_scalled = scaler.transform(df_numpy)
df_scalled = pd.DataFrame(df_scalled, columns=[
  'Opening', 'High', 'Low', 'Closing','Momentum', 'Range', 'ohlc'])

In [ ]:
df_train = df_scalled[2000:2801]
df_val = df_scalled[2801:3101] #300
df_game = df_scalled[3101:]

In [ ]:
lookback = 15
step = 1
delay = 0
batch_size = 128

In [ ]:
float_data = np.array(df_scalled).astype('float32')

In [ ]:
float_train_data = np.array(df_train).astype('float32')

In [ ]:
float_val_data = np.array(df_val).astype('float32')

In [ ]:
float_val_data[299]

array([0.66733915, 0.6516419 , 0.6893792 , 0.66730493, 0.5582318 ,
       0.13815914, 0.67210084], dtype=float32)

In [ ]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=1):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(
        min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][3] 
    yield samples, targets

In [ ]:
def generator2(data, delay, min_index, shuffle=False, batch_size=128, step=1):
  for i in range(10,299):
    lookback = i
    max_index = i
    rows = np.arange(i-10, min(i + batch_size, max_index))
    #i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][3] 
    yield samples, targets

In [ ]:
float_val_data.shape[0]

300

In [ ]:
def generator_custom(data, delay, min_index=0, shuffle=False, batch_size=128, step=1):
  samples = np.zeros((290,10,7)) # cannot make this variable
  targets = np.empty(shape=data.shape[0]-10)
  for i in range(10,300):
    max_index = i
    indices = range(0+i-10,max_index, step)
    
    samples[i-10] = data[indices]
    targets[i-10] = data[i][3]
  yield samples, targets

In [ ]:
try1 = generator_custom(float_val_data,
                        delay = 0)

In [ ]:
len(next(try1)[0])

290

In [ ]:
lol = (next(try1))

In [ ]:
type(try1)

generator

In [ ]:
train_gen_sample = generator(float_train_data,
    lookback = 10,
    delay=delay,
    min_index=0,
    max_index=300,
    #shuffle=True,
    step=step,
    batch_size=batch_size)

In [ ]:
next(train_gen_sample)

In [ ]:
next(val_gen)[0][0][0]

array([0.57219875, 0.55239791, 0.60156822, 0.5722214 , 0.36636439,
       0.04265104, 0.57693237])

In [ ]:
i = 10
train_gen_sample = generator(float_train_data,
    lookback=i,
    delay=delay,
    min_index=0,
    max_index=i,
    #shuffle=True,
    step=step,
    batch_size=batch_size)

In [ ]:
next(train_gen_sample)[0][0]

In [ ]:
gens = []
for i in range(10, 299):
  val_gen = generator(float_val_data,
    lookback=i,
    delay=delay,
    min_index=0,
    max_index=i+1,
    #shuffle=True,
    step=step,
    batch_size=batch_size)
  gens.append(val_gen)
  

In [ ]:
from itertools import chain

generator3 = chain(next(gens[0])[0], next(gens[1])[0])
for item in generator3:
    print(item)

In [ ]:
sum = np.array(generator3)

In [ ]:
sum

array(<itertools.chain object at 0x7fade62db490>, dtype=object)

In [ ]:
sum = combine()

In [ ]:
next(sum)

iterate through gens, take 1 generator each time and take out samples and targets, store in array - not possible!

In [ ]:
def refactor():
  somearr = np.empty(289, dtype=object)
  samples = np.empty(289)
  for i in range (0,289):
    samples = (next(gens[i])[0])
    somearr[i] = (samples)
    #targets[i] = (next(gens[i])[1])
  yield somearr #targets

In [ ]:
def show():
  for i in range (0,289):
    print(np.array(next(gens[i])[0][0], dtype=object))
    #print(np.array(next(gens[i])[1], dtype=object))
    break

In [ ]:
show()

In [ ]:
def refactor():
  for i in range (0,289):
    samples = np.arange((len(gens[i][0])),7)
    targets = np.arange(1)
    samples.append(next(gens[i])[0])
    targets.append(next(gens[i])[1])
  yield samples, targets

In [ ]:
def refactor3():
  for i, row in enumerate(gens):
    for j in enumerate(next(gens[i])[0]):
      print(next(gens[i])[0][j])
      break
    break 
    #yield samples, targets

In [ ]:
refactor3()

IndexError: ignored

delay must be 0 

In [ ]:
len(next(gens[250])[0][0])# działa, zbiór generatorów, każdy więcej o 1

7

In [ ]:
next(gens[0])[0][0]

array([[0.57219875, 0.55239791, 0.60156822, 0.5722214 , 0.36636439,
        0.04265104, 0.57693237],
       [0.58011794, 0.5624305 , 0.60343933, 0.58010876, 0.48002759,
        0.09311409, 0.58396685],
       [0.57691813, 0.55954635, 0.60382867, 0.57690722, 0.48690513,
        0.07320607, 0.58170485],
       [0.58391148, 0.5655309 , 0.61216742, 0.58391178, 0.44541064,
        0.06123606, 0.58882916],
       [0.58669031, 0.56946564, 0.61085874, 0.58669269, 0.43747884,
        0.09292509, 0.59092438],
       [0.59541613, 0.58252645, 0.61215663, 0.59540188, 0.49638233,
        0.16524917, 0.59901041],
       [0.60406029, 0.5798381 , 0.63770276, 0.60408509, 0.35380319,
        0.        , 0.60896665],
       [0.60806698, 0.58731616, 0.63429594, 0.60805094, 0.50094628,
        0.06558307, 0.61206889],
       [0.61158705, 0.59250754, 0.64072031, 0.61157423, 0.48879778,
        0.05991306, 0.61676526],
       [0.62215436, 0.60199416, 0.65060568, 0.62215298, 0.44599628,
        0.06035406, 0.6

In [ ]:
train_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=2501,
#shuffle=True,
step=step,
batch_size=batch_size)

In [ ]:
len(next(train_gen)[1])

128

In [ ]:
lolek = next(train_gen)

In [ ]:
type(lolek)

tuple

In [ ]:
val_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=2501,
max_index=3001,
step=step,
batch_size=batch_size)

NameError: ignored

In [ ]:
test_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=3001,
max_index=3543,
step=step,
batch_size=batch_size)

In [ ]:
train_steps = (2501 - lookback)
val_steps = (3001 - 2501 - lookback)
test_steps = (3543 - 3001 - lookback)

NameError: ignored

In [ ]:
model = tf.keras.models.Sequential()
model.add(layers.LSTM(120,
  activation='sigmoid',
  #dropout=0.1,
  #recurrent_dropout=0.1,
  input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(160,
  activation='sigmoid',))
model.add(layers.Dense(1,
  activation='sigmoid'))
model.compile(optimizer= tf.keras.optimizers.RMSprop(0.01), loss='mae')
history = model.fit(try1,
  steps_per_epoch=5,
  epochs=5)
  #validation_data=val_gen,
  #validation_steps=val_steps)

Epoch 1/5
5/5 [==============================] - 2s 7ms/step - loss: 0.2829
